### **1. Mask R-CNN image inference : practice**

In [ ]:
!mkdir ./image
!wget -O ./image/beatles01.jpg https://github.com/chulminkw/DLCV/blob/master/data/image/beatles01.jpg?raw=true

In [2]:
import cv2

image = cv2.imread('./image/beatles01.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [3]:
!mkdir ./pretrained
!wget -O ./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
!wget -O ./pretrained/config_mask_graph.pbtxt https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt

--2022-08-13 00:12:49--  http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 64.233.189.128, 2404:6800:4008:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|64.233.189.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177817887 (170M) [application/x-tar]
Saving to: ‘./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz’

./pretrained/mask_r 100%[===================>] 169.58M   113MB/s    in 1.5s    

2022-08-13 00:12:51 (113 MB/s) - ‘./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz’ saved [177817887/177817887]

--2022-08-13 00:12:51--  https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com 

In [4]:
!tar -xvf ./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz -C ./pretrained

mask_rcnn_inception_v2_coco_2018_01_28/
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
mask_rcnn_inception_v2_coco_2018_01_28/checkpoint
mask_rcnn_inception_v2_coco_2018_01_28/pipeline.config
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.meta
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/saved_model.pb
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/variables/
mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb


In [5]:
labels_to_names_seq = {0:'person',1:'bicycle',2:'car',3:'motorcycle',4:'airplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',
                       10:'fire hydrant',11:'street sign',12:'stop sign',13:'parking meter',14:'bench',15:'bird',16:'cat',17:'dog',18:'horse',19:'sheep',
                       20:'cow',21:'elephant',22:'bear',23:'zebra',24:'giraffe',25:'hat',26:'backpack',27:'umbrella',28:'shoe',29:'eye glasses',
                       30:'handbag',31:'tie',32:'suitcase',33:'frisbee',34:'skis',35:'snowboard',36:'sports ball',37:'kite',38:'baseball bat',39:'baseball glove',
                       40:'skateboard',41:'surfboard',42:'tennis racket',43:'bottle',44:'plate',45:'wine glass',46:'cup',47:'fork',48:'knife',49:'spoon',
                       50:'bowl',51:'banana',52:'apple',53:'sandwich',54:'orange',55:'broccoli',56:'carrot',57:'hot dog',58:'pizza',59:'donut',
                       60:'cake',61:'chair',62:'couch',63:'potted plant',64:'bed',65:'mirror',66:'dining table',67:'window',68:'desk',69:'toilet',
                       70:'door',71:'tv',72:'laptop',73:'mouse',74:'remote',75:'keyboard',76:'cell phone',77:'microwave',78:'oven',79:'toaster',
                       80:'sink',81:'refrigerator',82:'blender',83:'book',84:'clock',85:'vase',86:'scissors',87:'teddy bear',88:'hair drier',89:'toothbrush',
                       90:'hair brush'}

In [6]:
colors = list(
    [[0, 255, 0], 
     [0, 0, 255],
     [255, 0, 0],
     [0, 255, 255],
     [255, 255, 0],
     [255, 0, 255],
     [80, 70, 180],
     [250, 80, 190],
     [245, 145, 50],
     [70, 150, 250],
     [50, 190, 190]])

In [7]:
cv_net = cv2.dnn.readNetFromTensorflow('./pretrained/mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb', 
                                       './pretrained/config_mask_graph.pbtxt')

cv_net.setInput(cv2.dnn.blobFromImage(image, swapRB=True, crop=False))

boxes_out, masks_out = cv_net.forward(['detection_out_final', 'detection_masks'])
print(f'boxes_out shape : {boxes_out.shape} / masks_out shape : {masks_out.shape}')

boxes_detection_nums = boxes_out.shape[2]
masks_class_nums = masks_out.shape[1]
print(f'boxes_detection_nums : {boxes_detection_nums} / masks_class_nums : {masks_class_nums}')

boxes_out shape : (1, 1, 100, 7) / masks_out shape : (100, 90, 15, 15)
boxes_detection_nums : 100 / masks_class_nums : 90


In [8]:
boxes_out[:, :, 0, :]

array([[[0.        , 0.        , 0.99608445, 0.29211456, 0.41625595,
         0.46419758, 0.86386395]]], dtype=float32)

In [9]:
masks_out[0, 0, :, :]

array([[5.05761345e-05, 1.51209406e-05, 1.30301341e-05, 3.26758636e-05,
        1.25407096e-04, 1.59949542e-03, 1.72131639e-02, 9.73478034e-02,
        1.82075307e-01, 1.63195759e-01, 8.77476260e-02, 2.59182230e-02,
        3.98620404e-03, 5.24556846e-04, 4.42847231e-04],
       [7.68217706e-06, 3.63517438e-06, 5.46004685e-06, 4.70150226e-05,
        7.34756642e-04, 3.25893462e-02, 5.68581700e-01, 9.57904816e-01,
        9.86125648e-01, 9.72700715e-01, 8.33621740e-01, 3.00407857e-01,
        1.79045964e-02, 7.47650221e-04, 2.44054041e-04],
       [6.13494694e-06, 3.92934862e-06, 1.21724497e-05, 2.97153631e-04,
        6.27446454e-03, 1.55848145e-01, 8.96824658e-01, 9.92803276e-01,
        9.95611727e-01, 9.74057913e-01, 6.47798061e-01, 1.12822779e-01,
        6.54229242e-03, 1.94253167e-04, 5.53994687e-05],
       [1.16875181e-05, 9.90803346e-06, 5.14330895e-05, 2.26761447e-03,
        7.80595988e-02, 7.39753485e-01, 9.90540504e-01, 9.98218358e-01,
        9.96732950e-01, 9.16381538e-0

In [10]:
import numpy as np
import matplotlib.pyplot as plt

draw_image = image.copy()
image_height = draw_image.shape[0]
image_width = draw_image.shape[1]

confidence_threshold = 0.5
mask_threshold = 0.3

green_color=(0, 255, 0)
red_color=(0, 0, 255)

for i in range(boxes_detection_nums):
    box = boxes_out[0, 0, i, :]
    mask = masks_out[i, :, :, :]
    class_id = int(box[1])
    confidence_score = box[2]    
    if confidence_score > confidence_threshold:
        x1 = int(image_width * box[3])
        y1 = int(image_height * box[4])
        x2 = int(image_width * box[5])
        y2 = int(image_height * box[6])

        text = '{}: {:.4f}'.format(labels_to_names_seq[class_id], confidence_score)
        cv2.rectangle(draw_image, (x1, y1), (x2, y2), green_color, thickness=2)
        cv2.putText(draw_image, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, red_color, 1)

        # network model에서 detect된 object의 class mask 정보 추출
        class_mask_detected = mask[class_id]
        # 15*15 class mask 사이즈를 image에 맞추어 scale out 진행
        class_mask_scaled = cv2.resize(class_mask_detected, (x2-x1+1, y2-y1+1))
        # mask_threshold(0.3)에 따른 boolean 형태의 mask 정보 생성
        class_mask_scaled_bool = (class_mask_scaled > mask_threshold)

        # detect된 object의 mask에 랜덤 투명 색상 적용
        colorIndex = np.random.randint(0, len(colors)-1)
        color = colors[colorIndex]
        # mask를 적용할 image 내의 bounding box 영역 추출 
        before_masking_roi = draw_image[y1:y2+1, x1:x2+1]
        # bounding box 영역 내 mask 적용
        after_masking_roi = before_masking_roi
        after_masking_roi_bool = after_masking_roi[class_mask_scaled_bool]
        after_masking_roi[class_mask_scaled_bool] = ([0.3*color[0], 0.3*color[1], 0.3*color[2]]+0.6*after_masking_roi_bool).astype('uint8')

        # mask 정보를 이용해 contour 적용
        class_mask_scaled_int = class_mask_scaled_bool.astype('uint8')
        contours, hierarchy = cv2.findContours(class_mask_scaled_int, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(after_masking_roi, contours, -1, color, 1)

In [ ]:
plt.figure(figsize=(14, 16))
cv2.imwrite('./drive/MyDrive/Colab Notebooks/update/dl_cv/7_dl_cv_mask_rcnn_inference/data/beatles01_output.jpg', draw_image)
final_draw_image = plt.imread('./drive/MyDrive/Colab Notebooks/update/dl_cv/7_dl_cv_mask_rcnn_inference/data/beatles01_output.jpg')
plt.imshow(final_draw_image)
plt.show()

### **2. Mask R-CNN image inference : test**

In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
!wget -O ./image/baseball01.jpg https://raw.githubusercontent.com/chulminkw/DLCV/master/data/image/baseball01.jpg?raw=true

In [30]:
def get_box_info(box, image_width, image_height):
  class_id = int(box[1])
  x1 = int(image_width *box[3])
  y1 = int(image_height * box[4])
  x2 = int(image_width * box[5])
  y2 = int(image_height * box[6])
  return class_id, x1, y1, x2, y2

def draw_box(draw_image, box, image_width, image_height):
  green_color=(0, 255, 0)
  red_color=(0, 0, 255)
  confidence_score = box[2]    
  class_id, x1, y1, x2, y2 = get_box_info(box, image_width, image_height)
  text = '{}: {:.4f}'.format(labels_to_names_seq[class_id], confidence_score)
  cv2.rectangle(draw_image, (x1, y1), (x2, y2), green_color, thickness=2)
  cv2.putText(draw_image, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, red_color, 1)
  return draw_image

def draw_mask(draw_image, box, mask, image_width, image_height, mask_threshold):
  class_id, x1, y1, x2, y2 = get_box_info(box, image_width, image_height)
  class_mask_detected = mask[class_id]
  class_mask_scaled = cv2.resize(class_mask_detected, (x2-x1+1, y2-y1+1))
  class_mask_scaled_bool = (class_mask_scaled > mask_threshold)
  
  colorIndex = np.random.randint(0, len(colors)-1)
  color = colors[colorIndex]
  before_masking_roi = draw_image[y1:y2+1, x1:x2+1]
  after_masking_roi = before_masking_roi
  after_masking_roi_bool = after_masking_roi[class_mask_scaled_bool]
  after_masking_roi[class_mask_scaled_bool] = ([0.3 * color[0], 0.3 * color[1], 0.3 * color[2]] + 0.6 * after_masking_roi_bool).astype('uint8')

  class_mask_scaled_int = class_mask_scaled_bool.astype('uint8')
  contours, hierarchy = cv2.findContours(class_mask_scaled_int, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(after_masking_roi, contours, -1, color, 1)
  return draw_image

In [31]:
def detect_image_mask_rcnn(image, cv_net, confidence_threshold, mask_threshold, use_copied_image=True):
  if use_copied_image:
    draw_image = image.copy()
  else:
    draw_image = image

  image_height = draw_image.shape[0]
  image_width = draw_image.shape[1]
        
  start_time = time.time() 
  cv_net.setInput(cv2.dnn.blobFromImage(draw_image, swapRB=True, crop=False))
  boxes_out, masks_out = cv_net.forward(['detection_out_final', 'detection_masks'])
  inference_time = time.time() - start_time
  print(f'inference time : {np.round(inference_time)}초')

  boxes_detections_nums = boxes_out.shape[2]
  masks_classes_nums = masks_out.shape[1]

  for i in range(boxes_detections_nums):
    box = boxes_out[0, 0, i, :]
    mask = masks_out[i, :, :, :]
    confidence_score = box[2]
    if confidence_score > confidence_threshold:
      draw_box(draw_image, box, image_width, image_height)
      draw_mask(draw_image, box, mask, image_width, image_height, mask_threshold)
  return draw_image

In [32]:
image = cv2.imread('./image/baseball01.jpg')
cv_net = cv2.dnn.readNetFromTensorflow('./pretrained/mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb', './pretrained/config_mask_graph.pbtxt')
draw_image = detect_image_mask_rcnn(image, cv_net, confidence_threshold=0.3, mask_threshold=0.3, use_copied_image=True)

inference time : 4.0초


In [ ]:
plt.figure(figsize=(14, 16))
cv2.imwrite('./drive/MyDrive/Colab Notebooks/update/dl_cv/7_dl_cv_mask_rcnn_inference/data/baseball01_output.jpg', draw_image)
draw_image_final = plt.imread('./drive/MyDrive/Colab Notebooks/update/dl_cv/7_dl_cv_mask_rcnn_inference/data/baseball01_output.jpg')
plt.imshow(draw_image_final)
plt.show()